In [9]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from collections import defaultdict
from sklearn.model_selection import train_test_split, GridSearchCV
import nltk
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, f1_score, recall_score, confusion_matrix, classification_report, make_scorer
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import tree 
from xgboost import XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier, Booster
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import plotly.express as px

sns.set_style('whitegrid')

In [10]:
df=pd.read_csv('df_uda.csv', sep='\t')  

In [11]:
df.head()

,Unnamed: 0,hotel_name,review_text,review_data,link,page,review_rank_new,hotel_rank_new,new_rating,words,word_count,tokenized,tokenized_words,tokenized_v02
0,0,Pamookkoo Resort,I HAD A GREAT STAYING WITH THEM DURING MY VISI...,xan l wrote a review Feb 15,https://www.tripadvisor.com//Hotel_Review-g121...,0.0,5.0,4.5,excellent,"['I', 'HAD', 'A', 'GREAT', 'STAYING', 'WITH', ...",106,great stay visit phuket management helpful lea...,"['great', 'stay', 'visit', 'phuket', 'manageme...","['great', 'stay', 'visit', 'phuket', 'manageme..."
1,1,Pamookkoo Resort,Great place to stay! Everything is within walk...,poean wrote a review Dec 2022,https://www.tripadvisor.com//Hotel_Review-g121...,0.0,5.0,4.5,excellent,"['Great', 'place', 'to', 'stay!', 'Everything'...",85,great place stay everything within walk distan...,"['great', 'place', 'stay', 'everything', 'with...","['great', 'place', 'stay', 'everything', 'with..."
2,2,Pamookkoo Resort,This is by far the worst hotel i have stayed i...,Eva R wrote a review Feb 11,https://www.tripadvisor.com//Hotel_Review-g121...,0.0,1.0,4.5,poor,"['This', 'is', 'by', 'far', 'the', 'worst', 'h...",235,far bad hotel stay life hard describe dirty ar...,"['far', 'bad', 'hotel', 'stay', 'life', 'hard'...","['far', 'bad', 'hotel', 'stay', 'life', 'hard'..."
3,3,Pamookkoo Resort,"We are a family of 4, 2 adults and 2 teenagers...",ClareJens wrote a review Jan 2023,https://www.tripadvisor.com//Hotel_Review-g121...,0.0,2.0,4.5,poor,"['We', 'are', 'a', 'family', 'of', '4,', '2', ...",323,family adult teenager resort wrong choice fami...,"['family', 'adult', 'teenager', 'resort', 'wro...","['family', 'adult', 'teenager', 'resort', 'wro..."
4,4,Pamookkoo Resort,Heaven for family traveller's with kids.. it w...,RohitC1409 wrote a review Jan 2023,https://www.tripadvisor.com//Hotel_Review-g121...,0.0,5.0,4.5,excellent,"['Heaven', 'for', 'family', ""traveller's"", 'wi...",70,heaven family traveller kid like indian jone r...,"['heaven', 'family', 'traveller', 'kid', 'like...","['heaven', 'family', 'traveller', 'kid', 'like..."


In [13]:

#df.drop('Unnamed: 0', axis=1, inplace=True)

df = df[~df['review_text'].isnull()]

def preprocess(ReviewText):
    ReviewText = ReviewText.str.replace("(<br/>)", "")
    ReviewText = ReviewText.str.replace('(<a).*(>).*(</a>)', '')
    ReviewText = ReviewText.str.replace('(&amp)', '')
    ReviewText = ReviewText.str.replace('(&gt)', '')
    ReviewText = ReviewText.str.replace('(&lt)', '')
    ReviewText = ReviewText.str.replace('(\xa0)', ' ')  
    return ReviewText
df['Review Text'] = preprocess(df['review_text'])

#df['polarity'] = df['review_text'].map(lambda text: TextBlob(text).sentiment.polarity)
df['review_len'] = df['review_text'].astype(str).apply(len)
df['word_count'] = df['review_text'].apply(lambda x: len(str(x).split()))

/var/folders/58/7r05pkwd61v3p2z4n9krhd_r0000gp/T/ipykernel_73903/2903636058.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  ReviewText = ReviewText.str.replace("(<br/>)", "")
/var/folders/58/7r05pkwd61v3p2z4n9krhd_r0000gp/T/ipykernel_73903/2903636058.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  ReviewText = ReviewText.str.replace('(<a).*(>).*(</a>)', '')
/var/folders/58/7r05pkwd61v3p2z4n9krhd_r0000gp/T/ipykernel_73903/2903636058.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  ReviewText = ReviewText.str.replace('(&amp)', '')
/var/folders/58/7r05pkwd61v3p2z4n9krhd_r0000gp/T/ipykernel_73903/2903636058.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  ReviewText = ReviewText.str.replace('(&gt)', '')
/var/folders/58/7r05pkwd61v3p2z4n9krhd_r0000gp/T/ipykernel_73903/2903636058.p

In [23]:
import plotly.express as px

fig = px.histogram(df, x="review_len")
fig.show()

In [22]:

fig = px.histogram(df, x="word_count")
fig.show()